<img SRC="https://avatars2.githubusercontent.com/u/31697400?s=400&u=a5a6fc31ec93c07853dd53835936fd90c44f7483&v=4" WIDTH=125 ALIGN="right">

# Combining and splitting model layers

*D.A. Brakenhoff, Artesia, 2021*

This notebook shows methods for combining layers and splitting layers for MODFLOW models. Multiple layers can be combined into one layer or one layer can be split into sub-layers based on a fraction of the original thickness.


### Contents<a name="TOC"></a>
1. [Get data](#getdata)
2. [Split layers](#splitlayers)
3. [Combine layers](#combinelayers)

In [ ]:
import logging

import matplotlib.pyplot as plt
import nlmod
import numpy as np
import pandas as pd
from nlmod.visualise.netcdf import DatasetCrossSection
from shapely.geometry import LineString


In [ ]:
print(f'nlmod version: {nlmod.__version__}')

# toon informatie bij het aanroepen van functies
logging.basicConfig(level=logging.INFO)

## [1. Get data](#TOC)<a name="getdata"></a>

Define an extent to obtain REGIS

In [ ]:
extent = [131000, 136800, 471500, 475700]

Download and cache REGIS netCDF.

In [ ]:
ds = nlmod.read.regis.get_regis(extent)

Drop all non-existent layers in our area of interest.

In [ ]:
ds = nlmod.mgrid.set_idomain(ds, remove_nan_layers=True)

Let's take a look at the dataset

In [ ]:
ds

Define an line to draw a cross-section

In [ ]:
# diagonal line through extent
line = LineString([(extent[0], extent[2]), (extent[1], extent[3])])

Get colors for our cross-section plot.

In [ ]:
colors = nlmod.read.regis.get_legend()

Draw the cross-section for REGIS

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 6))
dcs = DatasetCrossSection(ds, line=line, top="top", bot="botm", ax=ax, zmin=-200, zmax=10)
dcs.plot_layers(colors=colors, min_label_area=1000)
dcs.plot_grid(linewidth=0.5, vertical=False)
ax.set_ylabel("m NAP")
ax.set_xlabel("Distance along x-sec (m)");

## [2. Split layers](#TOC)<a name="splitlayers"></a>

Define which layers you want to split and determine the indices for those layers

In [ ]:
split_layer_codes = ["PZWAz2", "PZWAz3"]
split_lays = np.argwhere(ds.layer.isin(split_layer_codes).data).squeeze()
split_lays

Next determine how to split the layers. This is done by creating a list of fractions that must add up to 1. The layer will be split into sub-layers from the top down, with each sub-layer getting a thickness equal to the fraction times the original thickness.

For example, `(0.5, 0.5)` will split the layer into two sub-layers, each getting a thickness equal to 50% of the original layer.

In [ ]:
# split dictionary
split_dict = {
    19: (0.3, 0.3, 0.4),
    20: (0.2, 0.2, 0.2, 0.2, 0.2)
}

Calculate the new layer elevations based on the information above.

In [ ]:
ds_split = nlmod.mdims.mlayers.split_layers_ds(ds, split_dict)

View the resulting Dataset:

In [ ]:
ds_split

The reindexer dictionary we stored links the new layer numbers to the old
layer numbers. This is convenient for copying data from the original layers to the new sub-layers.

In [ ]:
# 1st number = new layer index: should number continuously from 1..N
# 2nd number = old layer index: repeats where layer was split
ds_split.attrs["split_reindexer"]

The new layers are named according to the REGIS codes, with an added number if the layer was split into multiple sub-layers. For plotting a cross-section we want to use the original color from the REGIS cross-section plot for the new sub-layers.

In [ ]:
layer_names = []
colors_new = {}
for j, i in ds_split.split_reindexer.items():
    layercode = ds.layer.data[i]
    color_org = colors.at[layercode, "color"]
    if layercode in layer_names:
        suff = (
            np.sum([1 for ilay in layer_names if ilay.startswith(layercode)])
            + 1
        )
        layercode += f"_{suff}"
    layer_names.append(layercode)
    colors_new[layercode] = color_org


Plot the cross-section of the original and the new layer model.

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 12), sharex=True)
dcs2 = DatasetCrossSection(
    ds_split, line=line, top="top", bot="botm", ax=ax1, zmin=-200, zmax=10
)
polys2 = dcs2.plot_layers(colors=colors_new, min_label_area=1000)
dcs2.plot_grid(linewidth=0.5, vertical=False)
ax1.set_ylabel("m NAP")
ax1.set_title("Split layers")

dcs = DatasetCrossSection(
    ds, line=line, top="top", bot="botm", ax=ax2, zmin=-200, zmax=10
)
polys1 = dcs.plot_layers(colors=colors, min_label_area=1000)
dcs.plot_grid(linewidth=0.5, vertical=False)
ax2.set_ylabel("m NAP")
ax2.set_xlabel("Distance along x-sec (m)")
ax2.set_title("REGIS original")


## [3. Combine layers](#TOC)<a name="combinelayers"></a>

Example how to combine model layers. First find the indices of the layers to combine.

In [ ]:
combine_layers = [
    tuple(np.argwhere(ds.layer.str.startswith("URz").data).squeeze().tolist()),
    tuple(
        np.argwhere(ds.layer.isin(["PZWAz2", "PZWAz3"]).data)
        .squeeze()
        .tolist()
    ),
]
combine_layers


Combine layers using the `combine_layers_ds()` function and passing the layer dataset and the list of layer numbers to combine.

In [ ]:
ds_combine = nlmod.mdims.mlayers.combine_layers_ds(
    ds, combine_layers, kD=None, c=None
)


Take a look a the resulting dataset

In [ ]:
ds_combine

Plot the new and the old cross-section. Use layer code and color from first layer name for the combined layer

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 12), sharex=True)
dcs2 = DatasetCrossSection(
    ds_combine, line=line, top="top", bot="botm", ax=ax1, zmin=-200, zmax=10
)
polys2 = dcs2.plot_layers(colors=colors, min_label_area=1000)
dcs2.plot_grid(linewidth=0.5, vertical=False)
ax1.set_ylabel("m NAP")
ax1.set_title("Combine layers")

dcs = DatasetCrossSection(
    ds, line=line, top="top", bot="botm", ax=ax2, zmin=-200, zmax=10
)
polys1 = dcs.plot_layers(colors=colors, min_label_area=1000)
dcs.plot_grid(linewidth=0.5, vertical=False)
ax2.set_ylabel("m NAP")
ax2.set_xlabel("Distance along x-sec (m)")
ax2.set_title("REGIS original")
